Covid Stuff
====

Let's see if we can find any correlation between cases + deaths for the spread of covid in the UK

In [1]:
# Download the data

In [2]:
# Clean it and get it into a nice form

In [3]:
# Plot it

In [4]:
# Perform cross correlation

In [5]:
# Plot correlationi for waves 1 and 2